### SisPI data

In [190]:
from netCDF4 import Dataset
from pathlib import Path

import numpy as np


In [191]:
path = Path.joinpath(Path.cwd(), "data", "SisPI", "d03")
files_sispi = [f for f in path.iterdir() if f.name.startswith("wrfout") and not f.name.endswith(".dat")]

files_sispi.sort()

print(len(files_sispi))

files_sispi

24


[PosixPath('/home/jch/Documents/Grilla/Final MN/data/SisPI/d03/wrfout_d03_2021-07-06_01:00:00'),
 PosixPath('/home/jch/Documents/Grilla/Final MN/data/SisPI/d03/wrfout_d03_2021-07-06_02:00:00'),
 PosixPath('/home/jch/Documents/Grilla/Final MN/data/SisPI/d03/wrfout_d03_2021-07-06_03:00:00'),
 PosixPath('/home/jch/Documents/Grilla/Final MN/data/SisPI/d03/wrfout_d03_2021-07-06_04:00:00'),
 PosixPath('/home/jch/Documents/Grilla/Final MN/data/SisPI/d03/wrfout_d03_2021-07-06_05:00:00'),
 PosixPath('/home/jch/Documents/Grilla/Final MN/data/SisPI/d03/wrfout_d03_2021-07-06_06:00:00'),
 PosixPath('/home/jch/Documents/Grilla/Final MN/data/SisPI/d03/wrfout_d03_2021-07-06_07:00:00'),
 PosixPath('/home/jch/Documents/Grilla/Final MN/data/SisPI/d03/wrfout_d03_2021-07-06_08:00:00'),
 PosixPath('/home/jch/Documents/Grilla/Final MN/data/SisPI/d03/wrfout_d03_2021-07-06_09:00:00'),
 PosixPath('/home/jch/Documents/Grilla/Final MN/data/SisPI/d03/wrfout_d03_2021-07-06_10:00:00'),
 PosixPath('/home/jch/Document

In [192]:
D03_MINLON = 0
D03_MAXLON = 0
D03_MINLAT = 0
D03_MAXLAT = 0

In [193]:
rain, tmp = np.zeros(shape=(183, 411)), np.zeros(shape=(183, 411))
fwrf_lat, fwrf_lon = [], []

for fwrf in files_sispi:
    nc = Dataset(fwrf)

    print(f'Processing file: {fwrf}')

    base_path_results_sispi = str(
        Path.joinpath(fwrf.parent.parent.parent, 'sispy_results', fwrf.name))

    time = nc['Times']
    _date: str = ''.join([a.decode('UTF-8') for a in time[0][:]])

    rainc, rainnc = np.array(nc.variables['RAINC'][0]), np.array(nc.variables['RAINNC'][0])
    xlat, xlon = np.array(nc.variables['XLAT'][0]), np.array(nc.variables['XLONG'][0])

    if fwrf.name[22:24] == '00' and fwrf.name[19: 21] == _date[8: 10]:

        tmp = rainc + rainnc

        fwrf_lat = xlat
        fwrf_lon = xlon

        np.savetxt(base_path_results_sispi + "_LAT.dat", xlat, delimiter=',')
        np.savetxt(base_path_results_sispi + "_LON.dat", xlon, delimiter=',')

        D03_MINLAT = np.min(xlat)
        D03_MAXLAT = np.max(xlat)
        D03_MINLON = np.min(xlon)
        D03_MAXLON = np.max(xlon)

        continue
    else:
        rain = rainc + rainnc
        rain -= tmp
        tmp = rainc + rainnc

    np.savetxt(base_path_results_sispi + "_wrf.dat", rain, delimiter=',')


### GPM data

In [194]:
import h5py
import numpy as np
import matplotlib.pyplot as plt

from mpl_toolkits.basemap import Basemap
from mpl_toolkits.basemap import cm
from pathlib import Path

In [195]:
path = Path.joinpath(Path.cwd(), "data", "GPM")
files_gpm = [f for f in path.iterdir() if f.suffix == ".HDF5"]

files_gpm.sort()

files_gpm

[PosixPath('/home/jch/Documents/Grilla/Final MN/data/GPM/3B-HHR.MS.MRG.3IMERG.20210706-S000000-E002959.0000.V06B.HDF5'),
 PosixPath('/home/jch/Documents/Grilla/Final MN/data/GPM/3B-HHR.MS.MRG.3IMERG.20210706-S003000-E005959.0030.V06B.HDF5'),
 PosixPath('/home/jch/Documents/Grilla/Final MN/data/GPM/3B-HHR.MS.MRG.3IMERG.20210706-S010000-E012959.0060.V06B.HDF5'),
 PosixPath('/home/jch/Documents/Grilla/Final MN/data/GPM/3B-HHR.MS.MRG.3IMERG.20210706-S013000-E015959.0090.V06B.HDF5'),
 PosixPath('/home/jch/Documents/Grilla/Final MN/data/GPM/3B-HHR.MS.MRG.3IMERG.20210706-S020000-E022959.0120.V06B.HDF5'),
 PosixPath('/home/jch/Documents/Grilla/Final MN/data/GPM/3B-HHR.MS.MRG.3IMERG.20210706-S023000-E025959.0150.V06B.HDF5'),
 PosixPath('/home/jch/Documents/Grilla/Final MN/data/GPM/3B-HHR.MS.MRG.3IMERG.20210706-S030000-E032959.0180.V06B.HDF5'),
 PosixPath('/home/jch/Documents/Grilla/Final MN/data/GPM/3B-HHR.MS.MRG.3IMERG.20210706-S033000-E035959.0210.V06B.HDF5'),
 PosixPath('/home/jch/Documents/

In [196]:
GPM_NX = 1800
GPM_NY = 3600

rain_d03 = np.zeros(shape=(49, 118))
lats, lons = np.zeros(shape=(GPM_NX, GPM_NY)), np.zeros(shape=(GPM_NX, GPM_NY))

tmp_rain = np.zeros(shape=(GPM_NX, GPM_NY))

d03_lat, d03_lon = [], []
d03_lat_index, d03_lon_index = [], []

lonlat_save = 1
for indx, p_file in enumerate(files_gpm):
    gpm = str(p_file)
    base_path_results_gpm = str(
        Path.joinpath(p_file.parent.parent, 'gpm_results', p_file.name))

    print(f"Processing file: {gpm}")
    dataset = h5py.File(gpm, 'r')

    precip = np.transpose(dataset['Grid/precipitationCal'][:])

    if lonlat_save:
        lats = dataset['Grid/lat'][:]
        lons = dataset['Grid/lon'][:]

        for lat in lats:
            if D03_MINLAT <= lat <= D03_MAXLAT:
                d03_lat.append(lat)
                d03_lat_index.append(list(lats).index(lat))

        for lon in lons:
            if D03_MINLON <= lon <= D03_MAXLON:
                d03_lon.append(lon)
                d03_lon_index.append(list(lons).index(lon))

        np.savetxt(base_path_results_gpm + "_D03_LAT.dat", d03_lat, delimiter=',')
        np.savetxt(base_path_results_gpm + "_D03_LON.dat", d03_lon, delimiter=',')

        lonlat_save = 0

    if indx % 2 == 0 and indx != 0:
        rain_d03 = rain_d03 + precip[d03_lat_index[0]: d03_lat_index[-1], d03_lon_index[0]: d03_lon_index[-1], 0]
        np.savetxt(base_path_results_gpm + "_D03.dat", rain_d03, delimiter=',')

        fig = plt.figure(dpi=300)

        m = Basemap(projection='merc',
                    llcrnrlat=lats[d03_lat_index[0]],
                    urcrnrlat=lats[d03_lat_index[-1]],
                    llcrnrlon=lons[d03_lon_index[0]],
                    urcrnrlon=lons[d03_lon_index[-1]],
                    resolution='i')

        m.drawcoastlines()
        m.drawcountries()

        x, y = np.float32(
            np.meshgrid(lons[d03_lon_index[0]: d03_lon_index[-1]], lats[d03_lat_index[0]: d03_lat_index[-1]]))

        masked_array = np.ma.masked_where(rain_d03 < 0.1, rain_d03)

        cmap = cm.s3pcpn_l
        clevs = np.array([0.1, 0.6, 1, 2, 4, 6, 8, 12, 17, 21, 25, 33, 40, 50, 65, 80, 120, 150])

        cs = m.contourf(x, y, masked_array, clevs, cmap=cmap, latlon=True)

        parallels = np.arange(0., 90, 10)
        m.drawparallels(parallels, labels=[1, 0, 0, 0], fontsize=10)
        meridians = np.arange(180., 360., 10)
        m.drawmeridians(meridians, labels=[0, 0, 0, 1], fontsize=10)

        font = {'weight': 'bold', 'size': 6}
        plt.rc('font', **font)

        cbar = m.colorbar(cs, location='right', pad="5%")
        cbar.set_label('mm/h')

        plt.savefig(base_path_results_gpm + "_D03.png", dpi=300)
        plt.close()

        rain_d03[:, :] = 0.0

    else:
        rain_d03 = rain_d03 + precip[d03_lat_index[0]: d03_lat_index[-1], d03_lon_index[0]: d03_lon_index[-1], 0]

Processing file: /home/jch/Documents/Grilla/Final MN/data/GPM/3B-HHR.MS.MRG.3IMERG.20210706-S000000-E002959.0000.V06B.HDF5
Processing file: /home/jch/Documents/Grilla/Final MN/data/GPM/3B-HHR.MS.MRG.3IMERG.20210706-S003000-E005959.0030.V06B.HDF5
Processing file: /home/jch/Documents/Grilla/Final MN/data/GPM/3B-HHR.MS.MRG.3IMERG.20210706-S010000-E012959.0060.V06B.HDF5
Processing file: /home/jch/Documents/Grilla/Final MN/data/GPM/3B-HHR.MS.MRG.3IMERG.20210706-S013000-E015959.0090.V06B.HDF5
Processing file: /home/jch/Documents/Grilla/Final MN/data/GPM/3B-HHR.MS.MRG.3IMERG.20210706-S020000-E022959.0120.V06B.HDF5
Processing file: /home/jch/Documents/Grilla/Final MN/data/GPM/3B-HHR.MS.MRG.3IMERG.20210706-S023000-E025959.0150.V06B.HDF5
Processing file: /home/jch/Documents/Grilla/Final MN/data/GPM/3B-HHR.MS.MRG.3IMERG.20210706-S030000-E032959.0180.V06B.HDF5
Processing file: /home/jch/Documents/Grilla/Final MN/data/GPM/3B-HHR.MS.MRG.3IMERG.20210706-S033000-E035959.0210.V06B.HDF5
Processing file:

In [197]:
gpm_results_base_path = Path.joinpath(Path.cwd(), 'data', 'gpm_results')
sispi_results_base_path = Path.joinpath(Path.cwd(), 'data', 'sispy_results')
wrf2gpm_results_base_path = str(Path.joinpath(Path.cwd(), 'data', 'wrf2gpm_results'))

In [198]:
from scipy.interpolate import NearestNDInterpolator

for f in sispi_results_base_path.glob("*_wrf.dat"):
    rain_sispi = np.loadtxt(f, delimiter=',')

    points = np.concatenate((fwrf_lon.reshape(fwrf_lon.size, 1), fwrf_lat.reshape(fwrf_lat.size, 1)), axis=1)
    interp_val = NearestNDInterpolator(points, rain_sispi.reshape(rain_sispi.size, 1))

    x, y = np.float32(np.meshgrid(d03_lon, d03_lat))

    gpm_point = np.concatenate((x.reshape(x.size, 1), y.reshape(y.size, 1)), axis=1)

    rain_gpm = interp_val(gpm_point)

    np.savetxt(wrf2gpm_results_base_path + "/" + f.name + "_D03_wrf2gpm.dat",
               rain_gpm.reshape(len(d03_lat), len(d03_lon)),
               delimiter=',')

    # Plot the figure, define the geographic bounds
    fig = plt.figure(dpi=300)

    m = Basemap(projection='merc',
                llcrnrlat=d03_lat[0],
                urcrnrlat=d03_lat[-1],
                llcrnrlon=d03_lon[0],
                urcrnrlon=d03_lon[-1],
                resolution='i')

    m.drawcoastlines()
    m.drawcountries()

    x, y = np.float32(np.meshgrid(d03_lon, d03_lat))

    masked_array = np.ma.masked_where(rain_gpm < 0.1, rain_gpm)

    cmap = cm.s3pcpn_l
    clevs = np.array([0.1, 0.6, 1, 2, 4, 6, 8, 12, 17, 21, 25, 33, 40, 50, 65, 80, 120, 150])

    cs = m.contourf(x, y, masked_array.reshape(len(d03_lat), len(d03_lon)), clevs, cmap=cmap, latlon=True)

    parallels = np.arange(0., 90, 10)
    m.drawparallels(parallels, labels=[1, 0, 0, 0], fontsize=10)
    meridians = np.arange(180., 360., 10)
    m.drawmeridians(meridians, labels=[0, 0, 0, 1], fontsize=10)

    font = {'weight': 'bold', 'size': 6}
    plt.rc('font', **font)

    # Add colorbar

    cbar = m.colorbar(cs, location='right', pad="5%")
    cbar.set_label('mm/h')

    plt.savefig(wrf2gpm_results_base_path + "/" + f.name + "_D03_wrf2gpm.png", dpi=300)
    plt.close()

In [199]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import pandas as pd
import numpy as np

In [200]:
p = len(d03_lat) * len(d03_lon)
v_lat = np.repeat(d03_lat, p / len(d03_lat))
v_lon = np.repeat(d03_lon, p / len(d03_lon))

lat_d03_df, lon_d03_df, gpm_data, h_d03 = [], [], [], []

indx = 0

gpm_results = [x for x in gpm_results_base_path.iterdir() if x.is_file() and x.name.endswith("_D03.dat")]
gpm_results.sort()

for indx, f in enumerate(gpm_results):
    d = np.loadtxt(f, delimiter=',').flatten()

    for x in d:
        gpm_data.append(x)
        lat_d03_df.append(v_lat[indx % len(v_lat)])
        lon_d03_df.append(v_lon[indx % len(v_lon)])
        h_d03.append(indx)

        indx += 1

gpm = np.array(gpm_data)



In [201]:
p = len(fwrf_lat) * len(fwrf_lon)
v_lat = np.repeat(fwrf_lat, p / len(fwrf_lat))
v_lon = np.repeat(fwrf_lon, p / len(fwrf_lon))

sispi_results = [x for x in sispi_results_base_path.iterdir() if x.is_file() and x.name.endswith("_wrf.dat")]
sispi_results.sort()

lat_fwrf_df, lon_fwrf_df, h_fwrf, sispy_data = [], [], [], []

for indx, f in enumerate(sispi_results):
    d = np.loadtxt(f, delimiter=',').flatten()

    for x in d:
        sispy_data.append(1)
        lat_fwrf_df.append(v_lat[indx % len(v_lat)])
        lon_fwrf_df.append(v_lon[indx % len(v_lon)])
        h_fwrf.append(indx)

sispy = np.array(sispy_data)


In [202]:
df_d03 = pd.DataFrame({'lat': lat_d03_df, 'lon': lon_d03_df, 'prec': gpm_data, 'hrs': h_d03})
df_d03 = df_d03[df_d03['prec'] > 0]

print(len(lat_fwrf_df), len(lon_fwrf_df), len(sispy_data), len(h_fwrf))

df_fwrf = pd.DataFrame({'lat': lat_fwrf_df, 'lon': lon_fwrf_df, 'prec': sispy_data, 'hrs': h_fwrf})
df_fwrf = df_fwrf[df_fwrf['prec'] > 0]

x = np.column_stack((df_d03['lat'], df_d03['lon'], df_d03['hrs']))
y = df_d03['prec']

model = LinearRegression().fit(x, y)

print(model.score(x, y))
print(model.coef_)
print(r2_score(y, model.predict(x)))

1729899 1729899 1729899 1729899
0.05005352542137309
[ 7.34587889e+01  7.43377164e-01 -6.29629126e-02]
0.05005352542137309


In [203]:
print(len(sispy))

x_sispi = np.column_stack((df_fwrf['lat'], df_fwrf['lon'], df_fwrf['hrs']))

print(x_sispi.shape)

fixed_sispi_by_model = model.predict(x_sispi)

1729899
(1729899, 3)


In [204]:
mean = (sispy_data - fixed_sispi_by_model).sum() / len(sispy_data)

mean

-3.648573627648366